## import data set

In [1]:
! wget -q https://www.dropbox.com/s/1s26m9xbd6448ia/dataset.zip?dl=0

! unzip -q /content/dataset.zip?dl=0

In [2]:
# DAsta set source 
# https://metatext.io/datasets-list/summarization-task

In [3]:
! pip -q install rouge_score transformers datasets wandb accelerate

     |████████████████████████████████| 4.9 MB 5.2 MB/s 
     |████████████████████████████████| 432 kB 62.4 MB/s 
     |████████████████████████████████| 1.9 MB 51.6 MB/s 
     |████████████████████████████████| 148 kB 62.8 MB/s 
     |████████████████████████████████| 163 kB 62.5 MB/s 
     |████████████████████████████████| 6.6 MB 43.6 MB/s 
     |████████████████████████████████| 115 kB 41.7 MB/s 
     |████████████████████████████████| 212 kB 66.7 MB/s 
     |████████████████████████████████| 127 kB 63.3 MB/s 
     |████████████████████████████████| 182 kB 63.4 MB/s 
     |████████████████████████████████| 162 kB 65.2 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 162 kB 65.2 MB/s 
     |████████████████████████████████| 158 kB 67.5 MB/s 
     |████████████████████████████████| 157 kB 67.4 MB/s 
     |████████████████████████████████| 157 kB 67.7 MB/s 
     |████████████████████████████████| 157 kB 63.6 MB/s 
     |███████████

In [4]:
import warnings
warnings.filterwarnings('ignore')

import torch
import pandas as pd
import numpy as np
import transformers
import os
import re
from dataclasses import dataclass, field
from transformers import EncoderDecoderModel, Seq2SeqTrainer, Seq2SeqTrainingArguments
import datasets
import wandb



# load pretrained tokenizer 
tokenizer = transformers.RobertaTokenizerFast.from_pretrained("roberta-base")

# variables 
encoder_max_length = 256
decoder_max_length = 64
BATCH_SIZE = 32
EPOCHS = 30

if not os.path.exists("./best_model/"): 
    os.mkdir("./best_model/")
if not os.path.exists("./log_files/"):
    os.mkdir("./log_files/")

# log in to wandb
wandb.login() # eff91dd48ca6e7c55dfef9359e810cf3747cd6a2

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
# Download the dataset from the 
# from datasets import load_dataset, load_from_disk
# dataset = load_dataset("GEM/wiki_lingua") 
# dataset.save_to_disk("./data")

# convert the data to csv format
# os.mkdir("./dataset")
# [ data[i].to_csv(f"./dataset/{i}.csv") for i in os.listdir("./data/") if not i.endswith(".json") ] 


In [6]:
# Load the train dataset 
train = pd.read_csv("dataset/train.csv")
test = pd.read_csv("dataset/test.csv")
val = pd.read_csv("dataset/validation.csv")
train.head()

,Unnamed: 0,gem_id,gem_parent_id,source_language,target_language,source,target,references
0,0,wikilingua_multilingual-train-424377,wikilingua_multilingual-train-424377,en,en,Honesty is usually the best policy. It is disr...,"Tell the truth. Use a ""compliment sandwich"". B...","['Tell the truth. Use a ""compliment sandwich""...."
1,1,wikilingua_multilingual-train-424378,wikilingua_multilingual-train-424378,en,en,Go online and simply search “Decor Style Quizz...,Take a quiz online to find your style. Look on...,['Take a quiz online to find your style. Look ...
2,2,wikilingua_multilingual-train-424379,wikilingua_multilingual-train-424379,en,en,The size of a shuffleboard table can range any...,Determine what size you want your table to be....,['Determine what size you want your table to b...
3,3,wikilingua_multilingual-train-424380,wikilingua_multilingual-train-424380,en,en,"When you’re ready to start writing, dip the ni...","Dip the pen nib in the ink, then shake it firm...","['Dip the pen nib in the ink, then shake it fi..."
4,4,wikilingua_multilingual-train-424381,wikilingua_multilingual-train-424381,en,en,Being a man of your word means more than simpl...,Treat small promises with the same level of im...,['Treat small promises with the same level of ...


In [7]:
train = train.drop(labels= ['Unnamed: 0', 'gem_id', 'gem_parent_id', 'source_language', 'target_language', 'references'], axis =1  )
test = test.drop(labels= ['Unnamed: 0', 'gem_id', 'gem_parent_id', 'source_language', 'target_language', 'references'], axis =1  )
val = val.drop(labels= ['Unnamed: 0', 'gem_id', 'gem_parent_id', 'source_language', 'target_language', 'references'], axis =1  )
train.head()

,source,target
0,Honesty is usually the best policy. It is disr...,"Tell the truth. Use a ""compliment sandwich"". B..."
1,Go online and simply search “Decor Style Quizz...,Take a quiz online to find your style. Look on...
2,The size of a shuffleboard table can range any...,Determine what size you want your table to be....
3,"When you’re ready to start writing, dip the ni...","Dip the pen nib in the ink, then shake it firm..."
4,Being a man of your word means more than simpl...,Treat small promises with the same level of im...


In [8]:
print(f" size of the train data {train.shape[0]}")
print(f" size of the test data {test.shape[0]}")
print(f" size of the val data {val.shape[0]}")

 size of the train data 95517
 size of the test data 27489
 size of the val data 13340


In [9]:
# to get the breef over view of the data set 
n = np.random.randint(0 , train.shape[0] )

print( train.source.values[n] )
print("\n")
print( train.target.values[n] )

Lay your contact paper out beforehand and measure it. Use a ruler to measure the contact paper and mark the measurements with a pencil. Cut the sections of contact paper using scissors.  Lay the contact paper on the plywood square and use your fingers to stick it to the wood. Remove any air bubbles in the contact paper by rubbing it with your palms. The self-adhesive contact paper should stick to the plywood itself. Even if the contact paper seems secure, it might lift up and the puppies could chew on it. Use the staple gun to secure the contact paper to the floor of the whelping box. Staple as close to the sides of the floor as you can. Place staples between 2 different pieces of contact paper if you want extra security. Aim the staple gun so the staple pierces the edge of one piece and the edge of another, binding them. Make sure you line the strips up perfectly with the floor of the box. These strips will form the 3 walls of the whelping box. When you're finished, the 3 strips shoul

In [10]:
def traing_data_pre_processing(data):
    """
    input: 
    output: 
    """
    org_data_size = data.shape[0]

    data.dropna()

    for i, sent in enumerate( data.values ):
        source, target = sent

        # filter out the data that has target longer than the source
        if len( source.split(' ')) < len( target.split(' ')):
            data = data.drop(i, axis= 0) 

        # delete all the records with extremely short target 
        target = target.split(".")

        if target[-1] == "" or target[-1] == " ":
            target = target[:-1]
        
            
        count = 0
        for item in target:
            if len(item.split(" ")) <= 2:
                count += 1

        if count >= 2:
            data = data.drop(i, axis= 0) 
        else: 
            if len(target)  == 1 and len(target[0].split(" ")) <= 10:
                data = data.drop(i, axis= 0) 

    if org_data_size >  data.shape[0]:
        print(f"Number of records drop are {org_data_size - data.shape[0]}")

        
    return data.reset_index(drop= True)
    


train = traing_data_pre_processing(train)
test = traing_data_pre_processing(test)
val = traing_data_pre_processing(val)

print("\n")

print(f" size of the train data {train.shape[0]}")
print(f" size of the test data {test.shape[0]}")
print(f" size of the val data {val.shape[0]}")

Number of records drop are 3097
Number of records drop are 911
Number of records drop are 449


 size of the train data 92420
 size of the test data 26578
 size of the val data 12891


In [11]:
# drop all the records that has 

# org_data_size = train.shape[0]

# for i, sent in enumerate( train.source.values ): 
#     if len( tokenizer.encode(text = sent, add_special_tokens=False) ) > 512:
#         train.drop(i, inplace= True)  

# print(f"Number of records drop are {org_data_size - train.shape[0]}")

# # reset index
# train.reset_index(drop= True);


# make data loader 

In [12]:
# # To tokenize the data and create data loader 


# def text_tokenizer(data : list, max_length : int ):
#     """
#     input  :  list of code to be tokenized
#     output :  input_ids and attention_mask tensor of type long 
#     """
    
#     input_ids = []
#     attention_mask = [] 

#     for sent in data: 
#         encoded = tokenizer.encode_plus( text= str(sent).lower() , max_length= max_length, padding= 'max_length', truncation= True )
#         input_ids.append(encoded['input_ids'])
#         attention_mask.append(encoded['attention_mask'])


#     # covert the input ids and the attention mask to type tensor 
#     input_ids = torch.LongTensor(input_ids)
#     attention_mask = torch.LongTensor(attention_mask)

#     print(f"shape of the text input_ids -> {input_ids.shape}")
#     print(f"shape of the text attention_mask -> {attention_mask.shape}")

#     return  input_ids, attention_mask 


# def make_data_loader(data : pd.DataFrame):
#     """
#     input  : takes dataframe as a input 
#     output : outputs data loader iterator
#     """

#     # get the input ids and attention mask
#     SRC_input_ids, SRC_attention_mask  = text_tokenizer(data.source.values, 512)        # reurns input ids and attention mask of both source  -> shape [N, S] 
#     TGT_input_ids, TGT_attention_mask  = text_tokenizer(data.target.values, 256)        # reurns input ids and attention mask of both source  -> shape [N, T] 

#     # the pad index will be converted to -100 . this is helpfull in compute_metrics function below!! 
#     LABELS = torch.tensor( [ [ -100 if token == tokenizer.pad_token_id else token for token in t_ensor] for t_ensor in TGT_input_ids.tolist() ] ) # reurns  -> shape [N, T] 
#     # convert to tensordataser 
#     data = torch.utils.data.TensorDataset( SRC_input_ids, SRC_attention_mask, TGT_input_ids, TGT_attention_mask, LABELS ) # has four values in one variable 

#     sampler =  torch.utils.data.RandomSampler(data)

#     data_loader = torch.utils.data.DataLoader(dataset= data, batch_size= BATCH_SIZE, num_workers= 3, drop_last= True, sampler= sampler) # shape [N, S] -> [batchsize, sequence length]
    
#     return data_loader



# # Function call to create a data loader for thge model

# print("For train data")
# Train = make_data_loader(train)

# print("For validation data")
# Val = make_data_loader(val)

# print("For test data")
# Test = make_data_loader(test)

In [13]:
# in the data set form 
train_data = datasets.Dataset.from_pandas(train, preserve_index = False)
val_data = datasets.Dataset.from_pandas(val, preserve_index = False)
test_data = datasets.Dataset.from_pandas(test, preserve_index = False)

In [14]:
def process_data_to_model_inputs(batch):
  # tokenize the inputs and labels
  inputs = tokenizer(batch["source"], padding="max_length", truncation=True, max_length=encoder_max_length)
  outputs = tokenizer(batch["target"], padding="max_length", truncation=True, max_length=decoder_max_length)

  batch["input_ids"] = inputs.input_ids
  batch["attention_mask"] = inputs.attention_mask
  batch["decoder_input_ids"] = outputs.input_ids
  batch["decoder_attention_mask"] = outputs.attention_mask
  batch["labels"] = outputs.input_ids.copy()

  # because RoBERTa automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`. 
  # We have to make sure that the PAD token is ignored
  batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

  return batch

In [15]:
#processing training data
train_data = train_data.map(function= process_data_to_model_inputs, 
                            batched=True, 
                            batch_size=BATCH_SIZE, 
                            remove_columns=["source", "target"] )

train_data.set_format( type="torch", 
                    columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"])

#processing validation data
val_data = val_data.map( function= process_data_to_model_inputs, 
                            batched=True, 
                            batch_size=BATCH_SIZE, 
                            remove_columns=["source", "target"] )
val_data.set_format( type="torch", 
                        columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"] )


#processing test data
test_data = test_data.map( function= process_data_to_model_inputs, 
                            batched=True, 
                            batch_size=BATCH_SIZE, 
                            remove_columns=["source", "target"] )
test_data.set_format( type="torch", 
                        columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"] )

  0%|          | 0/2889 [00:00<?, ?ba/s]

  0%|          | 0/403 [00:00<?, ?ba/s]

  0%|          | 0/831 [00:00<?, ?ba/s]

# Develop Model

In [16]:
# defign the model 
model = EncoderDecoderModel.from_encoder_decoder_pretrained("roberta-base", "roberta-base", tie_encoder_decoder=True);

model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.encoder.pad_token_id = tokenizer.pad_token_id

# sensible parameters for beam search
# set decoding params    
model.config.early_stopping = True
model.config.max_length = 256
model.config.num_beams = 5
model.config.vocab_size = model.config.encoder.vocab_size
model.config.pad_token_id = tokenizer.pad_token_id


Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForCausalLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.encoder.layer.3.crossattention.self.value.bias', 'roberta.encoder.layer.4.crossattention.output.dense.bias', 'roberta.encoder.layer.2.crossattention.self.key.weig

In [17]:
%env WANDB_PROJECT= text_summarisation

env: WANDB_PROJECT=text_summarisation


In [18]:
model_arguments = Seq2SeqTrainingArguments(output_dir= "./best_model/",
                                            num_train_epochs = EPOCHS, 
                                            overwrite_output_dir= True, 
                                            do_train= True, 
                                            do_eval= True, 
                                            do_predict= True, 
                                            auto_find_batch_size= True, 
                                            evaluation_strategy = 'epoch',
                                            warmup_steps = 10000, 
                                            logging_dir = "./log_files/", 
                                            disable_tqdm = False, 
                                            load_best_model_at_end = True, 
                                            save_strategy= 'epoch', 
                                            save_total_limit = 1, 
                                            per_device_eval_batch_size= BATCH_SIZE, 
                                            per_device_train_batch_size= BATCH_SIZE, 
                                            predict_with_generate=True, 
                                            report_to='wandb',
                                            run_name="rober_based_encoder_decoder_text_summarisation",
                                            fp16=True ) # 

In [19]:

# load rouge for validation
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions
    
    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }
    

In [20]:
from transformers import EarlyStoppingCallback
from transformers.integrations import WandbCallback

In [21]:
early_stopping = EarlyStoppingCallback(early_stopping_patience= 5, 
                                        early_stopping_threshold= 0.001)

In [22]:
trainer = Seq2SeqTrainer(model = model, 
                            compute_metrics= compute_metrics,
                            args= model_arguments, 
                            train_dataset= train_data, 
                            eval_dataset= val_data, 
                            tokenizer=tokenizer, 
                            callbacks= [early_stopping, ]
                            )

Using cuda_amp half precision backend


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 92420
  Num Epochs = 30
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 86670
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


wandb: Currently logged in as: syedjunaidiqbal. Use `wandb login --relogin` to force relogin


You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
